In [ ]:
import itertools
import spotipy
import ytmusicapi
from tqdm import tqdm

In [ ]:
# export spotify client ID and client secret before executing this
auth_manager = spotipy.oauth2.SpotifyClientCredentials()
spotify_client=spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
# Read ytmusicapi documentation to copy headers
# https://ytmusicapi.readthedocs.io/en/stable/setup/browser.html
ytmusicapi.setup(filepath="headers_auth.json", headers_raw="""<headers copied>""")

In [ ]:
ytmusic_client = ytmusicapi.YTMusic("headers_auth.json")

In [ ]:
def playlist_items(spotify_playlist_id):
    for offset in itertools.count(step=100):
        resp = spotify_client.playlist_items(spotify_playlist_id, offset=offset, limit=100)
        for item in resp["items"]:
            yield item
        if resp["next"] is None:
            return

In [ ]:
items = list(playlist_items(spotify_playlist_id))

In [ ]:
item = items[0]

In [ ]:
title = item["track"]["name"]
album = item["track"]["album"]["name"]
artists = " ".join([_["name"] for _ in item["track"]["artists"] if _["type"] == "artist"])

query = f"{title} {album} {artists}"
query

In [ ]:
yt_playlist_id = ytmusic_client.create_playlist("<Playlist Name", "Playlist Description")

In [ ]:
search_results = ytmusic_client.search(query)

In [ ]:
video_ids = []

for item in tqdm(items):
    title = item["track"]["name"]
    album = item["track"]["album"]["name"]
    artists = " ".join([_["name"] for _ in item["track"]["artists"] if _["type"] == "artist"])

    query = f"{title} {album} {artists}"
    search_results = ytmusic_client.search(query)
    for result in search_results:
        if result["resultType"] == "song" or result["resultType"] == "video":
            video_ids.append(result["videoId"])
            break
    else:
        print(f"Could not find a match for {query}")

In [ ]:
ytmusic_client.add_playlist_items(yt_playlist_id, video_ids)
# You should see 'status': 'STATUS_SUCCEEDED' if successful